In [15]:
import numpy as np
from scipy.linalg import toeplitz
from scipy.sparse.linalg import cg
from numpy.linalg import norm, cond

A = np.array([ [1, 0.5, 0.25, 0.125, 0.0625],
              [0.5, 1, 0.5, 0.25, 0.125],
              [0.25, 0.5, 1, 0.5, 0.25],
              [0.125, 0.25, 0.5, 1, 0.5],
              [0.0625, 0.125, 0.25, 0.5, 1] ])

condition_number = cond(A, p=np.inf)
print(f"Число обумовленості матриці A: {condition_number}")
if round(condition_number) > 1:
    print("Матриця погано обумовлена.")
else:
    print("Матриця добре обумовлена.")



Число обумовленості матриці A: 7.499999999999999
Матриця погано обумовлена.


In [9]:
x_real = np.ones(n)
b1 = A @ x_real

def conjugate_gradient_method(A, b, x0=None, tol=1e-8, max_iter=1000):
    n_iters = [0]
    def callback(xk):
        n_iters[0] += 1
    
    if x0 is None:
        x0 = np.zeros(len(b))
    x, info = cg(A, b, x0=x0, rtol=tol, maxiter=max_iter, callback=callback)
    if info == 0:
        print(f"Метод спряжених градієнтів зійшовся за {n_iters[0]} ітерацій.")
    else:
        print("Метод спряжених градієнтів не зійшовся.")
    return x

def relaxation_method(A, b, omega=1.25, tol=1e-8, max_iter=1000):
    n = len(b)
    x = np.zeros(n)
    for iteration in range(max_iter):
        x_old = x.copy()
        for i in range(n):
            sigma = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x[i] = (1 - omega) * x[i] + (omega / A[i, i]) * (b[i] - sigma)
        if norm(x - x_old, ord=np.inf) < tol:
            print(f"Метод релаксації зійшовся за {iteration+1} ітерацій.")
            return x
    print("Метод релаксації не зійшовся.")
    return x

x_cg_b1 = conjugate_gradient_method(A, b1)
x_relax_b1 = relaxation_method(A, b1)

print("Розв'язок для b1 (метод спряжених градієнтів):", x_cg_b1)
print("Розв'язок для b1 (метод релаксації):", x_relax_b1)


Метод спряжених градієнтів зійшовся за 3 ітерацій.
Метод релаксації зійшовся за 23 ітерацій.
Розв'язок для b1 (метод спряжених градієнтів): [1. 1. 1. 1. 1.]
Розв'язок для b1 (метод релаксації): [1. 1. 1. 1. 1.]


In [12]:
epsilon = 0.001
b2 = b1 + epsilon * np.random.randn(n)
b3 = b1 - epsilon * np.random.randn(n)

x_cg_b2 = conjugate_gradient_method(A, b2)
x_relax_b2 = relaxation_method(A, b2)

x_cg_b3 = conjugate_gradient_method(A, b3)
x_relax_b3 = relaxation_method(A, b3)

print("\nПорівняння з реальним вектором x:")
print("Розв'язок для b1 (спряжені градієнти):", x_cg_b1 - x_real)
print("Розв'язок для b1 (релаксація):", x_relax_b1 - x_real)

print("Розв'язок для b2 (спряжені градієнти):", x_cg_b2 - x_real)
print("Розв'язок для b2 (релаксація):", x_relax_b2 - x_real)

print("Розв'язок для b3 (спряжені градієнти):", x_cg_b3 - x_real)
print("Розв'язок для b3 (релаксація):", x_relax_b3 - x_real)

# Висновки
print("\nВисновки:")
print("Чутливість методів до змін у правій частині (b) показує вплив погано обумовленої матриці A на стабільність розв'язку.")
print("Метод спряжених градієнтів може бути менш стійким у порівнянні з методом релаксації для таких некоректних матриць.")


Метод спряжених градієнтів зійшовся за 5 ітерацій.
Метод релаксації зійшовся за 23 ітерацій.
Метод спряжених градієнтів зійшовся за 5 ітерацій.
Метод релаксації зійшовся за 23 ітерацій.

Порівняння з реальним вектором x:
Розв'язок для b1 (спряжені градієнти): [-7.77156117e-16 -4.44089210e-16 -1.11022302e-15 -4.44089210e-16
 -7.77156117e-16]
Розв'язок для b1 (релаксація): [-1.27759137e-09 -2.25455021e-09  1.92982919e-09  3.82401666e-10
 -2.85376389e-10]
Розв'язок для b2 (спряжені градієнти): [-0.00075381  0.00151914 -0.00023413  0.0002397  -0.00136469]
Розв'язок для b2 (релаксація): [-0.00075381  0.00151913 -0.00023413  0.0002397  -0.00136469]
Розв'язок для b3 (спряжені градієнти): [-3.40123176e-04  1.51656084e-03 -4.38415285e-04 -4.03422411e-05
 -1.55566155e-03]
Розв'язок для b3 (релаксація): [-3.40124455e-04  1.51655859e-03 -4.38413356e-04 -4.03418601e-05
 -1.55566183e-03]

Висновки:
Чутливість методів до змін у правій частині (b) показує вплив погано обумовленої матриці A на стабільн